In [13]:
import dotenv
dotenv.load_dotenv('./azure.env')

from langchain.chat_models import AzureChatOpenAI
# gpt-35-turbo
chat = AzureChatOpenAI(temperature=0, azure_deployment='gpt-4')
chat.predict('''hello''')

'Hello! How can I assist you today?'

In [14]:
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.utilities import SerpAPIWrapper

In [15]:
import pickle

In [16]:
_uid = {
    '张三': 1000,
    '李四': 1001,
    '王五': 1002
}

@tool("query_uid")
def _query_uid(name: str) -> int:
    """根据姓名查询公司的员工编号

    Args:
        name (str): 中文姓名，姓在前，名在后，中间没有任何分隔

    Returns:
        int: 员工编号
    """
    return _uid[name]


_not_free = {
    (1000, 2023, 12, 1, 13), # 张三明天1300忙
    (1001, 2023, 12, 1, 14), # 李四明天1400忙
    (1002, 2023, 12, 1, 15), # 王五明天1500忙
}

@tool("query_calendar_free")
def _query_calendar_free(uid: int, year: int, month: int, day: int, hour_24: int) -> bool:
    """检查公司某位员工的日历在指定时间是否空闲。

    Args:
        uid (int): 员工编号
        year (int): 公历年号
        month (int): 公历月份
        day (int): 公历日
        hour_24 (int): 24小时制下的一天中的某个小时

    Returns:
        bool: true表示空闲
    """
    args = (uid, year, month, day, hour_24)
    return args not in _not_free


In [17]:
from datetime import datetime, timedelta

@tool("query_date")
def _query_date() -> dict:
    """查询今天的日期"""
    now = datetime.now()
    return {
        'year': now.year,
        'month': now.month,
        'day': now.day
    }


@tool("date_calc")
def _date_calc(year: int, month: int, day: int, delta_day: int) -> dict:
    """根据已有日期，和已知的偏移，计算得到一个新的日期。

    Args:
        year (int): 公元纪年
        month (int): 公历月份
        day (int): 公历日数
        delta_day (int): 新日期相对于今天的日数偏移。向未来偏移为正，向过去偏移为负。

    Returns:
        dict: 表示日期的json字典。
    """
    curr = datetime(year=year,month=month,day=day)
    delta = timedelta(days=delta_day)
    date_new = curr + delta
    # return date_new
    return {
        'year': date_new.year,
        'month': date_new.month,
        'day': date_new.day
    }
    

In [18]:
from langchain.prompts import PromptTemplate

In [19]:
react_tmpl_zh = PromptTemplate.from_template('''尽可能回答以下问题。 你可以使用以下工具：

{tools}

使用这些工具的方式是指定 json blob。
具体来说，这个 json 应该有一个"action"键（包含要使用的工具的名称）和一个"action_input"键（此处包含工具的输入）。

"action"字段的值**只能**从下列名称中选取：{tool_names}

$JSON_BLOB 应该只包含**单个**操作，**不要**返回多个操作的列表。 以下是有效 $JSON_BLOB 的示例：

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

你应当**始终**使用如下格式：

Question: 你必须回答的输入问题
Thought: 你应该时刻思考该做什么
Action:
```
$JSON_BLOB
```
Observation: 行动的结果
...（这个 Thought/Action/Observation 可以重复N次)
Thought: 我现在知道了最终答案
Final Answer: 原始输入问题的最终答案

开始！

Question: {input}
Thought: {agent_scratchpad}''')

In [20]:
print(prompt.format(input="我想明天下午和张三、王五开会。我约在几点合适？", agent_scratchpad=""))

NameError: name 'prompt' is not defined

In [21]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.utilities import SerpAPIWrapper

In [22]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser

In [23]:
tools = [_query_calendar_free, _query_date, _date_calc, _query_uid]
tools[0].run({'uid': 1, 'year': 1, 'month': 1, 'day': 1, 'hour_24': 1})

True

In [28]:
prompt = react_tmpl_zh.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

prompt
tools

[StructuredTool(name='query_calendar_free', description='query_calendar_free(uid: int, year: int, month: int, day: int, hour_24: int) -> bool - 检查公司某位员工的日历在指定时间是否空闲。\n\n    Args:\n        uid (int): 员工编号\n        year (int): 公历年号\n        month (int): 公历月份\n        day (int): 公历日\n        hour_24 (int): 24小时制下的一天中的某个小时\n\n    Returns:\n        bool: true表示空闲', args_schema=<class 'pydantic.main.query_calendar_freeSchemaSchema'>, func=<function _query_calendar_free at 0x7f2c97519af0>),
 StructuredTool(name='query_date', description='query_date() -> dict - 查询今天的日期', args_schema=<class 'pydantic.main.query_dateSchemaSchema'>, func=<function _query_date at 0x7f2c97519d30>),
 StructuredTool(name='date_calc', description='date_calc(year: int, month: int, day: int, delta_day: int) -> dict - 根据已有日期，和已知的偏移，计算得到一个新的日期。\n\n    Args:\n        year (int): 公元纪年\n        month (int): 公历月份\n        day (int): 公历日数\n        delta_day (int): 新日期相对于今天的日数偏移。向未来偏移为正，向过去偏移为负。\n\n    Returns:\n        dict: 表

In [25]:
chat_model_with_stop = chat.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

In [26]:
from langchain.agents import AgentType, Tool, initialize_agent

In [27]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [51]:
agent_executor.invoke({
    "input": "我想明天下午和张三、王五开会。我约在几点合适？"
})



> Entering new AgentExecutor chain...
Could not parse LLM output: 首先，我需要知道明天的日期。然后，我需要查询张三和王五的日程，看看他们明天下午有没有空。我将从查询明天的日期开始。Invalid or incomplete response我需要使用query_date工具来获取今天的日期。
Action:
```
{
  "action": "query_date",
  "action_input": {}
}
```{'year': 2023, 'month': 11, 'day': 30}今天是2023年11月30日。我需要使用date_calc工具来计算明天的日期。
Action:
```
{
  "action": "date_calc",
  "action_input": {"year": 2023, "month": 11, "day": 30, "delta_day": 1}
}
```{'year': 2023, 'month': 12, 'day': 1}明天是2023年12月1日。接下来，我需要查询张三和王五的员工编号。
Action:
```
{
  "action": "query_uid",
  "action_input": {"name": "张三"}
}
```1000张三的员工编号是1000。接下来，我需要查询王五的员工编号。
Action:
```
{
  "action": "query_uid",
  "action_input": {"name": "王五"}
}
```1002王五的员工编号是1002。接下来，我需要查询张三和王五明天下午的日程，看看他们有没有空。我将从13点开始查询，直到找到一个他们都有空的时间。
Action:
```
{
  "action": "query_calendar_free",
  "action_input": {"uid": 1000, "year": 2023, "month": 12, "day": 1, "hour_24": 13}
}
```False张三在13点没有空。我需要查询他在14点是否有空。
Action:
```
{
  "action": "query_calendar_free",
  

{'input': '我想明天下午和张三、王五开会。我约在几点合适？', 'output': '明天下午14点是合适的开会时间。'}

In [20]:
agent_executor.invoke({
    # "input": "我想明天下午和张三、王五开会，不过我下午2点之后才有空。我约在几点合适？"
    "input": "我想明天下午和张三、李四和王五开会。我约在几点合适？"
})



> Entering new AgentExecutor chain...
Could not parse LLM output: 首先，我需要知道明天的日期。然后，我需要查询张三、李四和王五的日程，看看他们明天下午有没有空。我可以先查询明天的日期。Invalid or incomplete response我需要使用query_date工具来获取今天的日期。
Action:
```
{
  "action": "query_date",
  "action_input": {}
}
```{'year': 2023, 'month': 11, 'day': 30}我现在知道了今天的日期是2023年11月30日。接下来，我需要计算明天的日期。我可以使用date_calc工具，将delta_day设置为1，来获取明天的日期。
Action:
```
{
  "action": "date_calc",
  "action_input": {"year": 2023, "month": 11, "day": 30, "delta_day": 1}
}
```{'year': 2023, 'month': 12, 'day': 1}我现在知道了明天的日期是2023年12月1日。接下来，我需要查询张三、李四和王五明天下午的日程。首先，我需要获取他们的员工编号。我可以使用query_uid工具来获取张三的员工编号。
Action:
```
{
  "action": "query_uid",
  "action_input": {"name": "张三"}
}
```1000我现在知道张三的员工编号是1000。接下来，我需要查询他明天下午的日程。我可以使用query_calendar_free工具，将小时设置为13到17，来查询他明天下午的日程。
Action:
```
{
  "action": "query_calendar_free",
  "action_input": {"uid": 1000, "year": 2023, "month": 12, "day": 1, "hour_24": 13}
}
```False张三在明天下午1点的时候不空闲。我需要继续查询他在明天下午其他时间的日程。
Action:
```
{
  "action": "query_ca

{'input': '我想明天下午和张三、李四和王五开会。我约在几点合适？', 'output': '明天下午4点是合适的会议时间。'}

In [52]:
agent_executor.invoke({
    # "input": "我想明天下午和张三、王五开会，不过我下午2点之后才有空。我约在几点合适？"
    "input": "我想明天下午和张三、李四和王五开会。我约在几点合适？"
})



> Entering new AgentExecutor chain...
首先，我需要知道今天的日期，然后我才能计算出明天的日期。然后，我需要查询张三、李四和王五的员工编号，以便我可以查看他们的日历。我将从查询今天的日期开始。 
Action:
```
{
  "action": "query_date",
  "action_input": {}
}
```{'year': 2023, 'month': 11, 'day': 30}现在我知道今天是2023年11月30日。接下来，我需要计算出明天的日期。
Action:
```
{
  "action": "date_calc",
  "action_input": {"year": 2023, "month": 11, "day": 30, "delta_day": 1}
}
```{'year': 2023, 'month': 12, 'day': 1}明天是2023年12月1日。接下来，我需要查询张三、李四和王五的员工编号。
Action:
```
{
  "action": "query_uid",
  "action_input": {"name": "张三"}
}
```1000张三的员工编号是1000。接下来，我需要查询李四的员工编号。
Action:
```
{
  "action": "query_uid",
  "action_input": {"name": "李四"}
}
```1001李四的员工编号是1001。接下来，我需要查询王五的员工编号。
Action:
```
{
  "action": "query_uid",
  "action_input": {"name": "王五"}
}
```1002王五的员工编号是1002。现在我有了所有人的员工编号，我可以开始检查他们明天下午的日程是否空闲。我将从下午1点开始检查，直到找到一个所有人都空闲的时间。
Action:
```
{
  "action": "query_calendar_free",
  "action_input": {"uid": 1000, "year": 2023, "month": 12, "day": 1, "hour_24": 13}
}
```False张三在下午1点有安排。我需要检查下一个小时。
A

{'input': '我想明天下午和张三、李四和王五开会。我约在几点合适？', 'output': '明天下午4点是所有人都空闲的时间，可以安排会议。'}